## ***SET UP AND PREPROCESS***

INSTALL

In [ ]:
!pip install sklearn-crfsuite

     |████████████████████████████████| 747kB 25.2MB/s 


In [ ]:
!pip install pythainlp

     |████████████████████████████████| 11.0MB 18.4MB/s 


In [ ]:
import json
from pythainlp.tag import pos_tag
from google.colab import drive
import sklearn_crfsuite
import sklearn_crfsuite.metrics
drive.mount('/content/drive')

Mounted at /content/drive


LOAD DATA

อย่ายุ่งกับส่วนนี้

In [ ]:
def load_data(file_path):
    data = open(file_path).read()
    list_all = data.split('\n')
    sentence_in_list_all = []
    sentence = []
    answer_isus = [] 
    Y = []
    X = []
    # print(list_all)
    for i in list_all:
      if i == '':
        sentence_in_list_all.append(sentence)
        sentence = []
      else:
        sentence.append(i)
    for sentences in sentence_in_list_all:
      answer = [] 
      keep_all_word_and_ner_each_sentence = []
      for len_word in range(len(sentences)):
        word_and_ner = []
        word_and_ner_split = sentences[len_word].split("\t")
        word_and_ner.append(word_and_ner_split[1][2:])
        word_and_ner.append(word_and_ner_split[0])
        keep_all_word_and_ner_each_sentence.append(word_and_ner)
      # print(keep_all_word_and_ner_each_sentence)

      replaced = ['x' if x[0]=='' else x for x in keep_all_word_and_ner_each_sentence]
      # print(replaced)
      keep = []
      for len_word_and_ner in range(len(replaced)):
        if replaced[len_word_and_ner] != 'x':
          if len_word_and_ner == len(replaced)-1:
            list_answer = []
            # print("1: add",replaced[len_word_and_ner][1])
            keep.append(replaced[len_word_and_ner][1])
            list_answer.append(replaced[len_word_and_ner][0])
            list_answer.append(''.join(keep))
            answer.append(tuple(list_answer))
            # print('KEEP:',keep)
          elif replaced[len_word_and_ner][0] == replaced[len_word_and_ner+1][0]:
            # print("2: add",replaced[len_word_and_ner][1])
            keep.append(replaced[len_word_and_ner][1])
            # print('KEEP:',keep)
          elif replaced[len_word_and_ner][0] != replaced[len_word_and_ner+1][0]:
            list_answer = []
            # print("3: add",replaced[len_word_and_ner][1])
            keep.append(replaced[len_word_and_ner][1])
            # print('KEEP:',keep)
            list_answer.append(replaced[len_word_and_ner][0])
            list_answer.append(''.join(keep))
            answer.append(tuple(list_answer))
            keep = []
        else:
          # print(4)
          continue
      # print(answer)
      answer_isus.append(answer) #เอาไป evaluate 
      list_tempX = []
      list_tempY = []
      for i in answer:
        list_tempX.append(i[1])
        list_tempY.append(i[0])
      X.append(list_tempX)
      # X.append(pos_tag(list_tempX, corpus='orchid_ud'))
      Y.append(list_tempY)
    return (X,Y,answer_isus)

ทดลอง B I I I มาต่อกัน

In [ ]:
def load_data(file_path):
    data = open(file_path).read()
    list_all = data.split('\n')
    sentence_in_list_all = []
    sentence = []
    answer_isus = [] 
    Y = []
    X = []
    # print(list_all)
    for i in list_all:
      if i == '':
        sentence_in_list_all.append(sentence)
        sentence = []
      else:
        sentence.append(i)
    for sentences in sentence_in_list_all:
      answer = [] 
      keep_all_word_and_ner_each_sentence = []
      for len_word in range(len(sentences)):
        word_and_ner = []
        word_and_ner_split = sentences[len_word].split("\t")
        word_and_ner.append(word_and_ner_split[1][:])
        word_and_ner.append(word_and_ner_split[0])
        keep_all_word_and_ner_each_sentence.append(word_and_ner)
      # print(keep_all_word_and_ner_each_sentence)

      replaced = ['x' if x[0]=='O' else x for x in keep_all_word_and_ner_each_sentence]
      # print(replaced)
      keep = []
      for len_word_and_ner in range(len(replaced)):
        if replaced[len_word_and_ner] != 'x':
          if len_word_and_ner == len(replaced)-1:
            list_answer = []
            # print("1: add",replaced[len_word_and_ner][1])
            keep.append(replaced[len_word_and_ner][1])
            list_answer.append(replaced[len_word_and_ner][0])
            list_answer.append(''.join(keep))
            answer.append(tuple(list_answer))
            # print('KEEP:',keep)


          # [['B_PER', 'ทดสอบ'],['B_PER', 'ธรรมนูญ'], ['I_PER', ' '], ['I_PER', 'ศรี'], ['I_PER', 'โรจน์'], 'x', 'x', 'x', 'x', ['B_NUM', '4']]

          elif (replaced[len_word_and_ner][0][2:] == replaced[len_word_and_ner+1][0][2:]): #ตรวจว่า PER == PER 
            if replaced[len_word_and_ner][0][0] == 'B' and replaced[len_word_and_ner+1][0][0] == 'I': #ตรวจว่าถ้าตอนนี้เป็น B และตัวถัดไปเป็น I ต้องเอามันมาติดกัน 
              keep.append(replaced[len_word_and_ner][1]) #เก็บคำของ B ไว้ก่อน
            elif (replaced[len_word_and_ner][0][0] == 'I' and replaced[len_word_and_ner+1][0][0] == 'I'):
              keep.append(replaced[len_word_and_ner][1])

            elif (replaced[len_word_and_ner][0][0] == 'I' and replaced[len_word_and_ner+1][0][0] == 'B'):
              list_answer = []
              keep.append(replaced[len_word_and_ner][1])
              list_answer.append(replaced[len_word_and_ner][0][2:])
              list_answer.append(''.join(keep))
              answer.append(tuple(list_answer))
              keep = []
            elif (replaced[len_word_and_ner][0][0] == 'B' and replaced[len_word_and_ner+1][0][0] == 'B'): #กรณีเป็นอย่างอื่นคืิอเราต้องแยกกัน
              keep = []
              list_answer = []
              keep.append(replaced[len_word_and_ner][1])
              list_answer.append(replaced[len_word_and_ner][0][2:])
              list_answer.append(''.join(keep))
              answer.append(tuple(list_answer))
              keep = []
          elif (replaced[len_word_and_ner][0][2:] != replaced[len_word_and_ner+1][0][2:]):
            list_answer = []
            keep.append(replaced[len_word_and_ner][1])
            list_answer.append(replaced[len_word_and_ner][0][2:])
            list_answer.append(''.join(keep))
            answer.append(tuple(list_answer))
            keep = []
        else:
          # print(4)
          continue
      # print(answer)
      answer_isus.append(answer) #เอาไป evaluate 
      list_tempX = []
      list_tempY = []
      for i in answer:
        list_tempX.append(i[1])
        list_tempY.append(i[0])
      # X.append(list_tempX)
      X.append(pos_tag(list_tempX, corpus='orchid_ud'))
      Y.append(list_tempY)
    return (X,Y,answer_isus)

In [ ]:
# load_data('earth.tsv')
# (earth1,earth2,answer) = 

# json.dump(answer, open('predicted_train_entities.json', encoding='utf8', mode='w'))

[['B_PER', 'ธรรมนูญ'], 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x']
[('PER', 'ธรรมนูญ')]
[['B_PER', 'ธรรมนูญ'], ['I_PER', ' '], ['I_PER', 'ศรี'], ['I_PER', 'โรจน์'], 'x', 'x', 'x', 'x', ['B_NUM', '4'], 'x', 'x', 'x', 'x', ['B_NUM', '68'], 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', ['B_MEA', '19'], ['I_MEA', ' '], ['I_MEA', 'ปี'], 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', ['B_NUM', '18'], 'x', 'x', 'x', 'x', 'x']
[('PER', 'ธรรมนูญ ศรีโรจน์'), ('NUM', '4'), ('NUM', '68'), ('MEA', '19 ปี'), ('NUM', '18')]


In [ ]:
earth1

[[('ธรรมนูญ', 'NOUN')],
 [('ธรรมนูญ ศรีโรจน์', 'NOUN'),
  ('4', 'NUM'),
  ('68', 'NUM'),
  ('19 ปี', 'NUM'),
  ('18', 'NUM')]]

Feature

In [ ]:
def pos_features(i, tokens):
    if i==0:
      return {'w0 pos': tokens[i][1]}
    if i==1:
      return {'w0 pos': tokens[i][1],'w-1 pos': tokens[i-1][1]}
    if i==len(tokens)-1:
      return {'w0 pos': tokens[i][1],'w-1 pos': tokens[i-1][1]}
    else:
      return {'w0 pos': tokens[i][1],'w-1 pos': tokens[i-1][1], 'w+1 pos': tokens[i+1][1]}
    
def pos_conjunctive_features(i, tokens):
  def conjunctive_pos(i, tokens, positions):
      position_str = ' '.join([str(x) for x in positions])
      words = []
      for position in positions:
          if 0 <= i + position < len(tokens): 
              words.append(tokens[i+position][1])
      return {'word i and word '+ position_str: tokens[i][1] + '_' + '_'.join(words)}
  return conjunctive_pos(i, tokens, [-1, 1])   

def word_features(i, tokens):
    if i==0:
      return {'w0 word': tokens[i][0]}
    if i==1:
      return {'w0 word': tokens[i][0],'w-1 word': tokens[i-1][0]}
    if i==len(tokens)-1:
      return {'w0 pos': tokens[i][0],'w-1 pos': tokens[i-1][0]}        
    else:    
      return {'w0 pos': tokens[i][0],'w-1 pos': tokens[i-1][0], 'w+1 pos': tokens[i+1][0]}



# ***TRAINING MODEL***

**เตรียมข้อมูล**

กรณีทำแค่ Train และ Dev
จะสามารถส่งข้อมูลเข้า load_data แล้วเทรนโมเดลได้เลยทันที

In [ ]:
X_train, Y_train,evaluate_train = load_data('/content/drive/MyDrive/CONTEST 3/Contest 3 - NER/train_auto_tok.tsv')
X_dev, Y_dev,evaluate_dev = load_data('/content/drive/MyDrive/CONTEST 3/Contest 3 - NER/dev_auto_tok.tsv')
# X_test, Y_test,evaluate_test= load_data('/content/drive/MyDrive/CONTEST 3/Contest 3 - NER/dev_auto_tok.tsv')

In [ ]:
X_train.pop()
Y_train.pop()
X_dev.pop()
Y_dev.pop()

[]

In [ ]:
def Union(lst1, lst2):
    final_list = lst1 + lst2
    return final_list

X_all = Union(X_train, X_dev)
Y_all = Union(Y_train, Y_dev)



In [ ]:
len(X_all)

68930

In [ ]:
len(Y_all)

68930

In [ ]:
json.dump(evaluate_train, open('predicted_train_entities.json', encoding='utf8', mode='w'))

In [ ]:
X_train[-5]

[('เวลาประมาณ 09.20 น.', 'NOUN'),
 ('นาย', 'NOUN'),
 ('รองพล เจริญพันธ์', 'PROPN'),
 ('ทำเนียบรัฐบาล', 'NOUN'),
 ('รัฐมนตรี', 'NOUN'),
 ('สำนักนายกรัฐมนตรี', 'NOUN'),
 ('นาย', 'NOUN'),
 ('เนวิน ชิดชอบ', 'PROPN'),
 ('สำนักนายกรัฐมนตรี', 'PROPN'),
 ('คณะปฏิรูปฯ', 'NOUN'),
 ('กองบัญชาการ', 'NOUN'),
 ('กองทัพบก', 'NOUN')]

In [ ]:
pip install attacut

     |████████████████████████████████| 1.3MB 16.5MB/s 
     |████████████████████████████████| 481kB 18.4MB/s 
     |████████████████████████████████| 92kB 9.2MB/s 
     |████████████████████████████████| 645kB 31.4MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=12f126cb195813b391fd30d81e42208249d9f884954f9598d56d41a127ffc305
  Stored in directory: /root/.cache/pip/wheels/af/19/30/1ea0cad502dcb4e66ed5a690279628c827aea38bbbab75d5ed
Successfully built fire
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from pythainlp.tokenize import word_tokenize
text = 'ณชพศ อินทรสมบัติ'
word_tokenize(text, engine="newmm")

['ณชพศ', ' ', 'อินทร', 'สมบัติ']

Model Functions

In [ ]:
def featurize_one_sentence(tokens, feature_function_list):
    feature_dict_seq = []
    for i in range(len(tokens)): # ทุก token
        feature_dict = {}
        
        for feature_fn in feature_function_list: # ทุก feature function

            feature_dict.update(feature_fn(i, tokens))
        feature_dict_seq.append(feature_dict)
    # print(feature_dict_seq)
    return feature_dict_seq
    
def train_and_evaluate(X_train, Y_train, X_test, feature_function_list):
    token_seq_list = X_train
    feature_seq_list = [featurize_one_sentence(token_list,feature_function_list) for token_list in token_seq_list]
    label_seq_list=Y_train
    #TODO: Fill this in
    crf = sklearn_crfsuite.CRF(verbose=True)
    crf.fit(feature_seq_list, label_seq_list)
    pred = crf.predict(X_test)
    
    # print(sklearn_crfsuite.metrics.flat_classification_report(label_seq_list, pred))
    
    return pred


def train_and_evaluate_butnofea(X_train, Y_train, X_test, Y_test):
    crf = sklearn_crfsuite.CRF(verbose=True)
    crf.fit(X_train, Y_train)
    pred = crf.predict(X_test)
    # print(sklearn_crfsuite.metrics.flat_classification_report(Y_test, pred))
    
    return pred


ห้องทดลอง

In [ ]:
X_train[-1]

[]

In [ ]:
import pandas as pd
df1 = pd.read_json ('/content/drive/MyDrive/CONTEST 3/Contest 3 - NER/test_entities.json')
df1.to_csv ('/content/drive/MyDrive/CONTEST 3/Contest 3 - NER/New File Name.txt', index = False)

In [ ]:
import pandas as pd
df = pd.read_json ('/content/drive/MyDrive/CONTEST 3/Contest 3 - NER/train_entities.json')
df.to_csv ('/content/drive/MyDrive/CONTEST 3/Contest 3 - NER/New File Name.txt', index = False)

In [ ]:
X_train

In [ ]:
len(X_all)

68930

In [ ]:
pred

In [ ]:
crf = sklearn_crfsuite.CRF()
# crf.fit(X_train, Y_train)
crf.fit(X_all, Y_all)
pred = crf.predict(list_test)
# print(pred)

In [ ]:
feature_seq_list = [featurize_one_sentence(token_list,[pos_features,pos_conjunctive_features,word_features]) for token_list in X_train]
crf = sklearn_crfsuite.CRF()

new = []
for len_sentence in range(len(X_train)):
  _temp6 = []
  for len_word in range(len(X_train[len_sentence])):
    l = list(X_train[len_sentence][len_word])
    # {'w0 word': 'เวลา 13.00 น.'}
    a = feature_seq_list[len_sentence][len_word]
    _list1 = list(a.items())
    
    # print(_list1)
    _x = []
    for w in _list1:
      for s in w:
        _x.append(s)

    z = l  + _x
    _temp6.append(tuple(z))
  new.append(_temp6)

# print(new[-2])
# print(len(new))
crf.fit(new, Y_train)
pred = crf.predict(X_test_2)


In [ ]:
new[-2]

In [ ]:
asasd = ('เวลา 13.00 น.', 'NOUN')


l = list(asasd)
l.append('x')

tuple(l)

In [ ]:
X_train[-2]

In [ ]:
feature_seq_list[-2]

# **Evaluate**

อันนี้เป็นการดึง test โดยจะดึงคำของ test มาแล้วใส่ pos ให้มัน

In [ ]:
import json
with open('/content/test_entities.json') as f:
  data = json.load(f)

# print(data)

list_test = []
for len_sentence in range(len(data)):
  _temp1 = []
  for len_word in range(len(data[len_sentence])):
    if len(data[len_sentence]) != 0:
      _temp1.append(data[len_sentence][len_word][1])
    else:
      continue

  list_test.append(_temp1)
  

In [ ]:
len(list_test)

5250

In [ ]:
from pythainlp.tag import pos_tag_sents

X_test_2 = pos_tag_sents(list_test , corpus='orchid_ud')

In [ ]:
list_test

อันนี้คือเวลาเรา predict แล้วจะเอาสิ่งที่เรา predict ได้มา link กับคำ match (หาที่อยู่)

In [ ]:
list_evaluate = []
for len_sentence in range(len(list_test)):
  temp = [] 
  for len_word in range(len(list_test[len_sentence])):
    temp2=[]
    temp2.append(pred[len_sentence][len_word])
    # print(X_test_2[len_sentence][len_word])
    temp2.append(list_test[len_sentence][len_word]) #กรณีไม่มีไรเลย
    # temp2.append(X_test_2[len_sentence][len_word][0]) #กรณีมี pos
    temp.append(tuple(temp2))
  list_evaluate.append(temp)

json.dump(list_evaluate, open('predicted_test_entities.json', encoding='utf8', mode='w'))

In [ ]:
X_all

In [ ]:
Y_all